In [1]:
pip install groq python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq
from getpass import getpass

# Securely ask for API key (only if not already set)
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    api_key = getpass("Enter your GROQ API Key: ")

client = Groq(api_key=api_key)

MODEL_NAME = "llama-3.3-70b-versatile"



MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a senior software engineer. "
            "Be precise, technical, and solution-oriented. "
            "Provide code examples when helpful."
        ),
        "temperature": 0.7,
    },
    "billing": {
        "system_prompt": (
            "You are a customer billing specialist. "
            "Be empathetic, polite, and policy-driven. "
            "Clearly explain refund policies and next steps."
        ),
        "temperature": 0.7,
    },
    "general": {
        "system_prompt": (
            "You are a helpful customer support assistant. "
            "Respond conversationally and clearly."
        ),
        "temperature": 0.7,
    },
}




def route_prompt(user_input: str) -> str:
    """
    Classifies the user query into one category:
    technical, billing, general, tool
    Returns ONLY the category name.
    """

    routing_prompt = """
    Classify the following text into one of these categories:
    [technical, billing, general, tool]

    - technical: programming, bugs, errors, code issues
    - billing: payments, refunds, subscriptions, charges
    - tool: requests for real-time data (e.g., price of Bitcoin)
    - general: casual chat or other inquiries

    Return ONLY the category name.
    """

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": routing_prompt},
            {"role": "user", "content": user_input},
        ],
    )

    category = response.choices[0].message.content.strip().lower()
    return category





def fetch_bitcoin_price():
    """
    Mock function to simulate tool usage.
    Replace with real API call if needed.
    """
    return "The current price of Bitcoin is $51,240 (mock data)."




def process_request(user_input: str) -> str:
    """
    1. Routes user input
    2. Selects correct expert
    3. Calls model OR tool
    4. Returns final response
    """

    category = route_prompt(user_input)
    print(f"[Router Decision]: {category}")

    # Tool handling
    if category == "tool":
        return fetch_bitcoin_price()

    # Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    config = MODEL_CONFIG[category]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=config["temperature"],
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input},
        ],
    )

    return response.choices[0].message.content





user_query = input("Ask something: ")
answer = process_request(user_query)
print("\nAssistant:", answer)


Enter your GROQ API Key: ··········
Ask something: I was charged twice for my subscription
[Router Decision]: billing

Assistant: I'm so sorry to hear that you were charged twice for your subscription. I can imagine how frustrating that must be for you. I'm here to help resolve this issue as quickly as possible.

To start, I'd like to apologize for the error and assure you that we'll do our best to correct it. Our policy is to ensure that our customers are only charged once for their subscription, and it's clear that we fell short of that standard in your case.

To proceed, I'll need to investigate this further and verify the duplicate charge. Could you please provide me with your subscription details, including your account name and the dates of the duplicate charges? This will help me to look into the matter and determine the best course of action.

Once I've verified the duplicate charge, I'll be happy to process a refund for the incorrect charge. Please note that our refund policy 